In [43]:
#即將出版
import requests
from bs4 import BeautifulSoup
import csv
import time
from datetime import datetime
import re  # 用於提取 bookID

# 分類頁面基礎URL模板，n 為 01 到 20，包含頁數參數
base_url_template = "https://www.books.com.tw/web/sys_compub/books/{:02d}/?page={}"

# 動態生成分類頁面的鏈結，n 從 01 到 20
categories = [n for n in range(1, 21)]

# 生成當天日期並加入到 CSV 文件名中
current_date = datetime.now().strftime('%m%d')
output_file = rf'C:\Users\USER\Desktop\books_data{current_date}.csv'

# CSV檔案的標題行，新增 'bookID'
csv_headers = ["bookID", "書名", "出版日期", "優惠價", "連結"]

# 每頁的書籍數量閾值，當書籍數量少於100時，表示已到最後一頁
PAGE_BOOK_THRESHOLD = 100

# 抓取單個分類頁面的書籍資訊
def scrape_books(category_num, page_num):
    url = base_url_template.format(category_num, page_num)
    print(f"正在抓取分類 {category_num} 第 {page_num} 頁：{url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 找到頁面中的書籍條目
    book_items = soup.select('div.grid_20.push_4.main_column div.mod_a li.item')
    
    if not book_items:  # 如果沒有書籍條目，說明已到最後一頁
        return None
    
    books_data = []
    
    for item in book_items:
        try:
            # 書名
            title = item.select_one('h4 a').text.strip()
            
            # 書籍鏈結
            book_link = item.select_one('h4 a')['href']
            full_link = book_link  # 完整的書籍鏈接
            
            # 提取 bookID (即 URL 中的數字部分)
            book_id = re.search(r'/products/(\d+)', full_link).group(1) if re.search(r'/products/(\d+)', full_link) else 'N/A'
            
            # 出版日期
            pub_date = item.select_one('ul.msg li.date').text.replace("出版日期：", "").strip()
            
            # 優惠價
            price_info = item.select_one('li.price_a').text
            discount_price = price_info.split("折")[1].strip()
            
            # 將 bookID 和其他數據一起保存
            books_data.append([book_id, title, pub_date, discount_price, full_link])
        
        except Exception as e:
            print(f"解析條目時出錯：{e}")
    
    return books_data

# 保存數據到CSV文件
def save_to_csv(data, file_name):
    with open(file_name, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(csv_headers)
        writer.writerows(data)

# 主程式
def main():
    all_books_data = []
    
    for category_num in categories:
        page_num = 1
        while True:
            books_data = scrape_books(category_num, page_num)
            if not books_data:
                break  # 如果該頁沒有書籍數據，則退出該分類的翻頁
            
            all_books_data.extend(books_data)
            
            # 監控進度
            print(f"從分類 {category_num} 的第 {page_num} 頁抓取了 {len(books_data)} 本書籍")
            
            # 如果當前頁書籍數量少於100，說明已到最後一頁，跳到下一分類
            if len(books_data) < PAGE_BOOK_THRESHOLD:
                break
            
            # 否則，繼續抓取下一頁
            page_num += 1
            time.sleep(2)  # 休眠一段時間，避免伺服器過載
    
    # 保存所有數據到CSV文件
    save_to_csv(all_books_data, output_file)
    print(f"數據已保存到 {output_file}")

# 執行主程式
if __name__ == "__main__":
    main()


正在抓取分類 1 第 1 頁：https://www.books.com.tw/web/sys_compub/books/01/?page=1
從分類 1 的第 1 頁抓取了 79 本書籍
正在抓取分類 2 第 1 頁：https://www.books.com.tw/web/sys_compub/books/02/?page=1
從分類 2 的第 1 頁抓取了 43 本書籍
正在抓取分類 3 第 1 頁：https://www.books.com.tw/web/sys_compub/books/03/?page=1
從分類 3 的第 1 頁抓取了 13 本書籍
正在抓取分類 4 第 1 頁：https://www.books.com.tw/web/sys_compub/books/04/?page=1
從分類 4 的第 1 頁抓取了 25 本書籍
正在抓取分類 5 第 1 頁：https://www.books.com.tw/web/sys_compub/books/05/?page=1
正在抓取分類 6 第 1 頁：https://www.books.com.tw/web/sys_compub/books/06/?page=1
從分類 6 的第 1 頁抓取了 7 本書籍
正在抓取分類 7 第 1 頁：https://www.books.com.tw/web/sys_compub/books/07/?page=1
從分類 7 的第 1 頁抓取了 27 本書籍
正在抓取分類 8 第 1 頁：https://www.books.com.tw/web/sys_compub/books/08/?page=1
從分類 8 的第 1 頁抓取了 11 本書籍
正在抓取分類 9 第 1 頁：https://www.books.com.tw/web/sys_compub/books/09/?page=1
從分類 9 的第 1 頁抓取了 7 本書籍
正在抓取分類 10 第 1 頁：https://www.books.com.tw/web/sys_compub/books/10/?page=1
從分類 10 的第 1 頁抓取了 10 本書籍
正在抓取分類 11 第 1 頁：https://www.books.com.tw/web/sys_compub/books/11/?page=1


In [42]:
# CSV 文件的路徑
csv_file = r"C:\Users\USER\Desktop\books_data1004.csv"

# 打開並讀取 CSV 文件
with open(csv_file, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    
    # 讀取 CSV 文件的每一行
    for row in csv_reader:
        print(row)

['bookID', '書名', '出版日期', '優惠價', '連結']
['0011001763', '喀邁拉空間：完美融合醫學懸疑與航太科技，醫學驚悚天后泰絲．格里森挑戰自我巔峰力作！', '2024/10/09', '504元', 'https://www.books.com.tw/products/0011001763?loc=P_0004_001']
['0011001765', '喀邁拉空間（博客來獨家版）', '2024/10/09', '504元', 'https://www.books.com.tw/products/0011001765?loc=P_0004_002']
['0011002233', '燕子不歸', '2024/10/15', '522元', 'https://www.books.com.tw/products/0011002233?loc=P_0004_003']
['0011002421', '神獸、怪物與人類：想像的極致，反映人心與社會價值的幻想動物', '2024/10/07', '720元', 'https://www.books.com.tw/products/0011002421?loc=P_0004_004']
['0011002245', '陰影下的陽光（限量親簽版）', '2024/10/05', '270元', 'https://www.books.com.tw/products/0011002245?loc=P_0004_005']
['0011002433', '浮士德博士（翻譯家彭淮棟最後譯作。德文直譯，單冊經典回歸版）', '2024/10/09', '891元', 'https://www.books.com.tw/products/0011002433?loc=P_0004_006']
['0011002042', '天才與瘋子的狂想', '2024/10/08', '315元', 'https://www.books.com.tw/products/0011002042?loc=P_0004_007']
['0011002226', '我在北京送快遞：那些失意，都很偉大', '2024/10/15', '342元', 'https://www.books.com.tw/products/001

['\ufeffbookID', '書名', '出版日期', '優惠價', '連結']
['0011001763', '喀邁拉空間：完美融合醫學懸疑與航太科技，醫學驚悚天后泰絲．格里森挑戰自我巔峰力作！', '2024/10/09', '504元', 'https://www.books.com.tw/products/0011001763?loc=P_0004_001']
['0011001765', '喀邁拉空間（博客來獨家版）', '2024/10/09', '504元', 'https://www.books.com.tw/products/0011001765?loc=P_0004_002']
['0011002233', '燕子不歸', '2024/10/15', '522元', 'https://www.books.com.tw/products/0011002233?loc=P_0004_003']
['0011002421', '神獸、怪物與人類：想像的極致，反映人心與社會價值的幻想動物', '2024/10/07', '720元', 'https://www.books.com.tw/products/0011002421?loc=P_0004_004']
['0011002245', '陰影下的陽光（限量親簽版）', '2024/10/05', '270元', 'https://www.books.com.tw/products/0011002245?loc=P_0004_005']
['0011002433', '浮士德博士（翻譯家彭淮棟最後譯作。德文直譯，單冊經典回歸版）', '2024/10/09', '891元', 'https://www.books.com.tw/products/0011002433?loc=P_0004_006']
['0011002042', '天才與瘋子的狂想', '2024/10/08', '315元', 'https://www.books.com.tw/products/0011002042?loc=P_0004_007']
['0011002226', '我在北京送快遞：那些失意，都很偉大', '2024/10/15', '342元', 'https://www.books.com.tw/produc